### 修改处
1. Group和TutorialGroup 里面group_id groupmate两者顺序不同，虽然无伤大雅，但是保持顺序一致更好
2. 读取csv文件建议整合成一个函数
3. 计算权重建议整合成一个函数
4. 根据PEP 8 的编程规范，类名使用驼峰命名法，函数名变量名使用下划线命名法

导入库

In [1]:
import random
import math
import copy
from collections import defaultdict # 可以不用吗？一共就那么几个属性，提前设定好不行？
import csv

# 全局变量
TG_NUM = 120
TG_SIZE = 50
GP_SIZE = 5
PROP_NUM = 3

定义类

In [ ]:
# 同学
class Student:
    def __init__(self, tutorial_group, student_id, school, name, gender, cgpa):
        self.tutorial_group = tutorial_group
        self.student_id = student_id
        self.school = school
        self.name = name
        self.gender = gender
        self.cgpa = float(cgpa)

# 小组 5人
class Group:
    def __init__(self,group_id):
        self.group_id = group_id # 建议两个类统一写
        self.groupmate = []
        self.score = 0

# 大组 50人
class TutorialGroup:
    def __init__(self, group_id):
        self.group_id = group_id
        self.groupmate = []  # Student对象列表
        self.male_ratio = 0
        self.schools = {}
        self.avg_cgpa = 0
        self.weights = {}    # 存放每组的熵权 {'School':x,'Gender':y,'CGPA':z}



In [30]:
# 计算每个小组的指标，方便分析初始数据
def calc_ratio(group:list):
    male_count = 0
    total_cgpa = 0
    schools = {}
    num_of_student = len(group)
    for student in group:
        total_cgpa += student.cgpa
        schools[student.school] = schools.get(student.shool, 0) + 1
        male_count += student.gender == 'Male'
    male_ratio = male_count / num_of_student
    avg_cgpa = total_cgpa / num_of_student
    return male_ratio, list(schools), avg_cgpa

def calc_ratio_dataset(dataset:dict):
    for tutorial_group in dataset.values():
        tutorial_group.male_ratio, tutorial_group.schools, tutorial_group.avg_cgpa = calc_ratio(tutorial_group.groupmate)

In [31]:

# 读取文件单独一个函数
def read_csv(file_path:str)->dict:
    dataset = {}   # {组号: TutorialGroup 对象}
    with open(file_path, newline='', encoding='utf-8') as f: # 防止空行，保证格式正确
        reader = csv.DictReader(f) # 读取csv文件，并自动转成字典 
        for row in reader:
            
            new_student = Student(
                tutorial_group = row["Tutorial Group"],
                student_id = row["Student ID"],
                school = row["School"],
                name = row["Name"],
                gender = row["Gender"],
                cgpa = row["CGPA"], 
            )
            if new_student.tutorial_group not in dataset:#存组号
                dataset[new_student.tutorial_group] = TutorialGroup(new_student.tutorial_group) # 创建组存组号     
            dataset[new_student.tutorial_group].groupmate.append(new_student) # 选组，找组员，添加
    calc_ratio_dataset(dataset)
    return dataset

In [32]:
# dataset 结构是什么？
dataset = read_csv("records.csv")
# print(dataset)
gp = dataset['G-1']
st_name = [st.name for st in gp.groupmate]
print(st_name)
print(len(gp.schools))


AttributeError: 'Student' object has no attribute 'shool'

In [ ]:
# 熵权法计算权重
# 标准化数据后，根据比例计算熵，根据熵的比例计算权重

def entropy_weight(data_matrix):
    n = len(data_matrix) # 学生数量
    m = len(data_matrix[0]) # 指标数量
    norm = []
    for j in range(m):
        col = [data_matrix[i][j] for i in range(n)]
        min_val, max_val = min(col), max(col)
        norm_col = [(x - min_val) / (max_val - min_val + 1e-9) for x in col] 
        norm.append(norm_col) # 熵权法标准化
    norm_T = list(zip(*norm)) # 转置为每行单位为学生存放数据

    P = []  # 比例矩阵 P
    for j in range(m):
        col = [norm_T[i][j] for i in range(n)]
        col_sum = sum(col) + 1e-9 
        P.append([x / col_sum for x in col])

    E = [] # 熵 E
    for j in range(m):
        e = -sum([p * math.log(p + 1e-9) for p in P[j]]) / math.log(n)
        E.append(e)

    D = [1 - e for e in E] # 冗余度 D & 权重 W
    W = [d / sum(D) for d in D]
    return W

#文字指标转换为数字计算权重
def encode_category(value, mapping_dict):
    if value not in mapping_dict:
        mapping_dict[value] = len(mapping_dict) + 1#编码
    return mapping_dict[value]

# 计算权重也应当单独开一个函数
def compute_weights(dataset:dict):
    school_map, gender_map = {}, {}
    for group_id, tg in dataset.items():
        data_matrix = []
        for student in tg.groupmate:
            school_code = encode_category(student.school, school_map)
            gender_code = encode_category(student.gender, gender_map)
            cgpa_value = student.cgpa
            data_matrix.append([school_code, gender_code, cgpa_value]) # 文字指标转数字
        weights = entropy_weight(data_matrix)
        tg.weights = {"School": weights[0],"Gender": weights[1],"CGPA": weights[2]}

In [10]:
compute_weights(dataset)
print(dataset['G-104'].weights)

{'School': 0.38934880270768746, 'Gender': 0.5110811022874541, 'CGPA': 0.09957009500485842}


贪心算法部分

伪代码:
```  
FUNCTION calc_score_v1(the list of 5 student, which tutorial group are they from):  
    quanzhong = the previous calculated weights
    
    calculate the gender_score:
        iterate through the members of the five_person group and find the current number of males
        calculate the current proportion of males in the five-person group
        iterate through the members of the tutorial group and find the current number of males
        calculate the current proportion of males in the tutorial group
        the gender_score = the absolute value of proportion's differnce

    calculate the school_diversity:
        find the current number of schools of the group
        find the current number of student of the group
        school_diversity = 1-school_number/student_number

    caculate the cgpa_score:
        calculate the average cgpa of the tutorial group
        calculate the average cgpa of the group
        cgpa_score = the absolute value of (average_cgpa_in_group - average_cgpa_in_tutorialgroup)

    caculate the score:
        score = gender_score*the_weight_of_gender_score + school_diversity*the_weight_of_school_diversity + cgpa_score*the_weigh_of_cgpa_score
```

In [ ]:
# TODO: 这个函数干的事情太多了，是否前期处理数据的时候，除了计算所谓熵权之外，也应该分别写三个函数计算性别比例，院校数量与平均成绩？
# 并把这三个数据作为TG的一个特征放到类里面？
# 如果given_list里面没有人，就直接在最开始返回0即可，或者干脆避免向这个函数里传入空列表


def calc_score(given_list, tutorialgroup_id):
    if not given_list:
        return 0
    weight = dataset[tutorialgroup_id].weights
    

    #计算性别均衡性分数
    #计算当前小组性别比
    
    
    #计算Tutorial group的性别比
    male_count = 0
    for student in dataset[tutorialgroup_id].groupmate:
        if student.gender == 'Male':
            male_count += 1
    target_male_ratio = male_count / TG_SIZE # 这里一组50人，不应该除以120
    gender_score = abs(male_ratio - target_male_ratio)

    #计算专业均衡性分数
    school_in_group = len(set(stu.school for stu in given_list)) # 
    school_diversity = school_in_group / len(given_list)
    school_score = 1-school_diversity 
    
    #计算CGPA均衡性分数
    total_cgpa = 0
    for student in dataset[tutorialgroup_id].groupmate:
        total_cgpa += student.cgpa
    target_average_cgpa = total_cgpa / TG_SIZE # 老错误，一组只有五十人
    current_total_cgpa = 0

    for student in given_list:
        current_total_cgpa += student.cgpa
    current_average_cgpa = current_total_cgpa / len(given_list)
    cgpa_score = abs(current_average_cgpa - target_average_cgpa)
    #根据权重计算总分
    score = gender_score * weight['Gender'] + school_score * weight['School'] + cgpa_score * weight['CGPA']
    return score

交换退火

交换函数

伪代码：  
FUNCTION exchange_two(groups):  
    ID1,ID2 = randomly choose 2 groups from group  
    pick1 = selected group1  
    pick2 = selected group2  
    student1,student2 = randomly choose 1 student from each selected group  
    exchange student1 and student2  
    RETURN pick1,pick2,student1,student2

In [33]:
# 非全局变量名称应当小写
# 传入的是一个字典吧，名字改成dict更好

def calc_score_after_total(id1, id2, tutorialgroup_list, tutorialgroup_id):
    return calc_score(tutorialgroup_list[id1].groupmate,tutorialgroup_id) + calc_score(tutorialgroup_list[id2].groupmate,tutorialgroup_id)

def exchange_two(groups, tutorial_group_id, number_of_groups):
    id1, id2 = random.sample(range(number_of_groups), 2)  #随机选取两个组
    pick1 = groups[id1]
    pick2 = groups[id2]  #定义pick1 pick2为选出的两个组
    previous_score = calc_score_after_total(id1, id2, groups, tutorial_group_id)
    student1 = random.randint(0, len(pick1.groupmate)-1)
    student2 = random.randint(0, len(pick2.groupmate)-1)  #在两个组里随机各选一个人
    pick1.groupmate[student1],pick2.groupmate[student2] = pick2.groupmate[student2],pick1.groupmate[student1]
      #交换两个人
    return id1,id2,previous_score  #这个返回什么后面看需求吧

计算交换后的分数

判断是否接受

伪代码:
FUNCTION accept_change(delta,temperature):  
    IF delta > 0:
        RETURN True
    ELSE:
        sta = exp(delta/temperature)
        IF:
            random_value < sta
                RETURN TRUE
        ElSE:
            RETURN False            

In [34]:
def accept_change(delta,temperature):  #delta和temperature在后面退火的函数再说，delta就是分数差
    if delta > 0:
        return True   #交换后变好，直接接受
    else:
        sta = math.exp(delta/temperature)
        return random.random() < sta   #变坏后有一定几率接受

In [35]:
# 建议把number_of_groups 先写死成全局变量 GP_NUM, 日后如需要交互式输入再修改

def stratified_grouping(current_tutorialgroup, group_size, number_of_groups):
    tutorialgroup_students = dataset[current_tutorialgroup].groupmate 
    # students格式：[student1,student2,......,student50]
    # 创建分层分组字典stratified_groups，格式:{('学院','性别'):[student1,student2,......],......}
    stratified_groups = defaultdict(list)
    for student in tutorialgroup_students:
        key = (student.school, student.gender)
        stratified_groups[key].append(student)
    # 每一层按成绩从高到低排序
    for key in stratified_groups:
        stratified_groups[key].sort(key=lambda x: x.cgpa, reverse=True)
    groups = {i: [] for i in range(1, number_of_groups+1)} #储存分组结果的字典
    groups_score = [0] * (number_of_groups + 1) # 我个人认为，组号可以从0-9而不一定非得是1-10.大不了最后导出的时候所有组号再统一加一就好

    # 开始按稀有度分组
    sorted_layers = sorted(stratified_groups.items(), key=lambda x:len(x[1])) #此处x：(('学院','性别'),[student1,student2,...])；按稀有度排序
    count = 0 #记录分组次数，次数达到10（分组数）后退出稀有度分组
    # 这个名字与你之前的students重复了，建议改
    # 疑惑：如果sorted_layers内部元素数量小于你的分组数，那至少有一组最开始分不到人
    # 因为你每一类最多只取出了一个同学
    for layer_key, students in sorted_layers:
        if count >= number_of_groups:
            break
        if students: #该层非空：
            target = min(groups.keys(), key = lambda x: len(groups[x])) #寻找空的组，获取其编号
            target_student = students.pop(0) #从该层中拿走成绩最好的学生
            #stratified_groups[layer_key].remove(target_student)
            groups[target].append(target_student) #将该学生加入小组
            count += 1
            #print(f"将层级{layer_key}的学生分到第{target}组")
    #更新分数
    for i in range(1,number_of_groups+1):
        groups_score[i] = calc_score(groups[i], current_tutorialgroup)
    #开始按贪心算法分组，不再需要层级
    remaining_students = []
    # TODO: 之前已经有一个组记录了全部的学生，现在又要重新记录一遍，空间上可以优化
    for layer_key, students in sorted_layers:
        remaining_students.extend(students)

    copied_remaining_students = copy.copy(remaining_students)
    for student in copied_remaining_students:
        best_group = None
        lowest_score_increase = 13000721 #计算分数增加的最小值 这个数的来源是什么？如果是要一个很大的数可以用float('inf‘)
        best_group_score = None
        full = True #通过布尔值记录是否所有小组已满
        for i in range(1,number_of_groups+1):
            # 这里面可以单独写一个函数，专门用来计算最适合的组
            if len(groups[i]) >= group_size:
                continue
            full = False
            #生成临时小组，计算分数增加量
            former_score = groups_score[i]
            # TODO：直接复制太占用空间，建议先加入再取出，记录过程中分数最高的，然后再真正加进去
            temp_group = copy.copy(groups[i])
            temp_group.append(student)
            current_score = calc_score(temp_group, current_tutorialgroup)
            # 可否直接记录尝试过程中的最小分数？以及产生最小分数的那个组？
            score_increase = current_score - former_score
            if score_increase < lowest_score_increase:
                lowest_score_increase = score_increase
                best_group = i
                best_group_score = current_score
            #print(f"第{i}组分数增加量为{score_increase}")
        # print(f"选中小组{best_group}，分数增加量为{lowest_score_increase}")
        if not full:
            remaining_students.remove(student)
            groups[best_group].append(student)
            groups_score[best_group] = best_group_score
        # print(f"将{(student.school,student.gender)}添加到第{best_group}组")
        member_num = [len(groups[i]) for i in range(1,number_of_groups+1)] # 11 是不合理的
        # print(member_num)
    if full: # 如果有剩下的学生 是随机分配对吧？
        randomlist = random.sample(range(1,number_of_groups+1),len(remaining_students))
        for i,student in enumerate(remaining_students):
            groups[randomlist[i]].append(student)
    # 贪心分组完成
    # 坏了，没有用Group类，现在加上 
    # TODO：如果用不上这个类其实可以不用 如果要用建议从最开始就用
    output_groups = [Group(i) for i in range(1,number_of_groups+1)]
    for i in range(1,number_of_groups+1):
        output_groups[i-1].groupmate = groups[i]
        output_groups[i-1].score = groups_score[i]
        # print(f"第{i}小组有{len(groups[i])}人")
    return output_groups

退火主函数

伪代码：  
FUNCTION annealing(groups,tutorialgroup_id,initial_temp=100 (#initial temperature),cooling_rate=0.99 (#the speed of cooling),min_tem (#min temperature,when temperature<min_temperature,break),max_iter=1000 (#it can iterate at most 1000 times)):  
    current_groups = copy_of_groups
    current_score = the_group_score_obtained_from_the_previous_process  
    initialize temperature = 0  
    initialize iteration = 0  
    initialize the num of consecutive exchanges that are all not accepted = 0  
    WHILE temperature is not below min_tem AND iteration has not exceeded max_iter:  
        iteration = iteration+1  
        initialize new_groups = current_groups  
        initialize new_score = current_score  
        use exchange_two to exchange two student  
        calculate new_score of the tutorial group  
        caculate the score difference before and after  
        IF change is accepted:  
            current_groups = new_groups  
            currrent_score = new_score  
            let no_change_count = 0 again
        ELSE:  
            no_change_count = no_change_count +1   
    temperature cool down  
    RETURN current_groups,current_score      

In [36]:
def annealing(tutorialgroup_id,group_size,number_of_groups,initial_tem=100,cooling_rate=0.99,min_tem=0.01, max_iter=1000):  #这些都是自己定义的数据，后面商量一下定位多少
    #ori_groups = copy.deepcopy(groups)  #储存原始组，如果不接受交换还能找到
    #ori_score = calc_score_after_total(groups)
    current_groups = stratified_grouping(tutorialgroup_id,group_size,number_of_groups)  #初始化 # 建议当成一个参数传到函数里面，不要在函数里面算
    temperature = initial_tem
    iteration = 0  #计算迭代次数
    no_change_count = 0  #计算连续交换后都不接受的情况发生的次数
    while temperature > min_tem and iteration < max_iter:
        iteration += 1
        new_groups = copy.deepcopy(current_groups)
        id1,id2,current_score = exchange_two(new_groups,tutorialgroup_id,number_of_groups)#交换
        new_score = calc_score_after_total(id1,id2,new_groups,tutorialgroup_id)
        delta = current_score - new_score  #计算调换后的分数和delta 
        if accept_change(delta,temperature):
            current_groups = new_groups
            #current_score = new_score  #也就是说。ori_groups保留原始数据永远不动，new_groups储存交换之后的分组信息，不管交换是否被接受，current_groups在确定交换接受后把这个新的分组储存进去
            no_change_count = 0 #因为是连续不接受，所以一旦接受就重新变成0
        else:
            no_change_count += 1
        temperature *= cooling_rate  #每次循环后降温一次
    return current_groups

主程序

In [37]:
def main():
    try:
        group_size = 5
        # int(input("Group size: "))
        number_of_groups = 50 // group_size 
    except:
        print("Invalid Input")
        return None
    file = open("output.csv","w",newline = '',encoding="utf-8")
    writer = csv.writer(file)
    header = ['Tutorial Group', 'Group', 'Student ID', 'School', 'Name', 'Gender', 'CGPA']
    writer.writerow(header)
    # global dataset
    for tutorialgroup_id, tutorial_group in dataset.items():  # ('G-1',TutorialGroup(1))
        output_groups = annealing(tutorialgroup_id,group_size,number_of_groups)
        for i,group in enumerate(output_groups):
            for student in group.groupmate:
                row = [tutorialgroup_id,i+1,student.student_id,student.school,student.name,student.gender,student.cgpa]
                writer.writerow(row)
        score_list = [calc_score(output_groups[i].groupmate, tutorialgroup_id) for i in range(number_of_groups)]
        print(f"第{tutorialgroup_id}大组，每组分数：{score_list}")
    file.close()
main()

第G-1大组，每组分数：[0.016953523584485538, 0.01705054703972694, 0.0847723180075312, 0.10406998463971884, 0.09912178842241223, 0.01234490946052356, 0.010501463810938983, 0.010646998993800708, 0.015983289032072495, 0.11624377783463806]
第G-10大组，每组分数：[0.05949309903288482, 0.05296447702644043, 0.07954872525851138, 0.05174036040023205, 0.10331364197457185, 0.07730451144379606, 0.0805688224470183, 0.05235241871333624, 0.10719001129089828, 0.08118088076012231]
第G-100大组，每组分数：[0.08941497779179022, 0.09078936872668039, 0.025161927199233693, 0.08641958285708501, 0.022622353232853123, 0.03037130969437301, 0.021645594015014463, 0.020603717515986716, 0.027441032040857034, 0.031413186193400756]
第G-101大组，每组分数：[0.06830039398872088, 0.06291201386958194, 0.06376281073049851, 0.06858399294235974, 0.06291201386958156, 0.06518080549869268, 0.0648972065450537, 0.06518080549869268, 0.06319561282322092, 0.06858399294235937]
第G-102大组，每组分数：[0.02645841976029788, 0.09636810157106344, 0.026500026694232504, 0.100528794964526